# Scratch Artificial Neural Network

In [1]:
from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.dont_write_bytecode = True

# Import libraries
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms


from ArtificialNeuralNetwork import ArtificialNeuralNetwork
from Layer import Layer, OutputLayer
from Function import ActivationFunction, LossFunction
from enums import InitializerType

In [11]:
# Disable __grad based on assistant direction
torch.set_grad_enabled(False)
# Verify MPS support
if torch.backends.mps.is_available():
    print("M1 GPU is available and PyTorch is configured to use MPS!")
else:
    print("MPS backend is not available. Check your setup.")

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

M1 GPU is available and PyTorch is configured to use MPS!


# Pipeline for testing MNIST dataset

In [4]:
input_size = 784
hidden_layers = 0
hidden_size = 20
output_size = 10
learning_rate = 0.001
param_1 = 0
param_2 = 0

In [5]:
# Import Dataset
train = pd.read_csv("data/train.csv")

In [12]:
 # Define a simple transformation.
transform = transforms.Compose([transforms.ToTensor()])

# Load MNIST training and test datasets.
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Create data loaders.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [13]:
ann = ArtificialNeuralNetwork(
    123,
    Layer(
        weight_init=InitializerType.RANDOM_DIST_UNIFORM,
        bias_init=InitializerType.ZERO,
        input_size=input_size,
        num_neurons=hidden_size,
        param_1=param_1,
        param_2=param_2,
        activation=ActivationFunction.relu,
        layer_name=f"Hidden Layer 0"
    ),
    *[Layer(
        weight_init=InitializerType.RANDOM_DIST_UNIFORM,
        bias_init=InitializerType.ZERO,
        input_size=hidden_size,
        num_neurons=hidden_size,
        param_1=param_1,
        param_2=param_2,
        activation=ActivationFunction.relu,
        layer_name=f"Hidden Layer {i + 1}"
    ) for i in range(hidden_layers)],
    OutputLayer(
        weight_init=InitializerType.RANDOM_DIST_UNIFORM,
        bias_init=InitializerType.ZERO,
        input_size=hidden_size,
        num_neurons=output_size,
        param_1=param_1,
        param_2=param_2,
        activation=ActivationFunction.relu,
        layer_name="Output Layer"
    )
)

In [15]:
ann.train(
    train_loader,
    lr=0.05,
    epochs=20,
    loss_function=LossFunction.categorical_cross_entropy
)

Epoch 1: Loss = 11.1094
Epoch 2: Loss = 11.0242
Epoch 3: Loss = 10.9381
Epoch 4: Loss = 10.9565
Epoch 5: Loss = 10.7558
Epoch 6: Loss = 10.9889
Epoch 7: Loss = 10.9081
Epoch 8: Loss = 11.5288
Epoch 9: Loss = nan
Epoch 10: Loss = nan
Epoch 11: Loss = nan
Epoch 12: Loss = nan
Epoch 13: Loss = nan
Epoch 14: Loss = nan
Epoch 15: Loss = nan
Epoch 16: Loss = nan
Epoch 17: Loss = nan


KeyboardInterrupt: 

In [16]:
ann.test(test_loader)

Test Accuracy: 9.80%
